In [ ]:
# Import Libraries
import time

from ImportData2D import get_label, getFilename, get_file_name
from stlSlicer import getSlicer, slicecoor, rotatestl
from imgSave import saveplot
import numpy as np

np.random.seed(125)


In [ ]:
label = getLabel("Occ_Sum", "median", True, False,False)
print(type(label[0]))
label = getLabel("Occ_Sum", "median", True, True,False)
print(type(label[0][0]))

In [ ]:
import os
test_name = "prep_test.tfrecords"
test_name = os.path.join("data", test_name)
print(test_name)

In [ ]:
import os
A = './data/preparation_181_data_test_eval_address.txt'
print(os.path.abspath(A))
print(os.path.isfile(A))

In [ ]:
example_grouped_address = ['/home/pasin/Documents/Google_Drive/Aa_TIT_LAB_Comp/Library/Tooth/Tooth/Model/my2DCNN/data/cross_section/PreparationScan_0_84140','/home/pasin/Documents/Google_Drive/Aa_TIT_LAB_Comp/Library/Tooth/Tooth/Model/my2DCNN/data/cross_section/PreparationScan_1_95138_0','/home/pasin/Documents/Google_Drive/Aa_TIT_LAB_Comp/Library/Tooth/Tooth/Model/my2DCNN/data/cross_section/PreparationScan_1_304138_45']
grouped_address = [1,2,3]
# define an empty list
train_address = []

# open file and read the content in a list
with open(A, 'r') as filehandle:  
    for line in filehandle:
        # remove linebreak which is the last character of the string
        current_name = line[:-1]
        # print(current_name)
        for i, name in enumerate(example_grouped_address):
            
            if current_name in name:
                train_address.append(grouped_address[i])
                grouped_address.remove(grouped_address[i])
                example_grouped_address.remove(example_grouped_address[i])
                

In [ ]:
print(train_address)

In [ ]:
import ast
B = ast.literal_eval(error_file_names[0])
print(B[0])

In [ ]:
train_address = ['abc_0_0','abc_0_90','abc_0_45','abc_0_135','abc_1_0','abc_1_45','abc_1_90','abc_1_70']

In [ ]:
train_address.sort()
print([s for s in train_address if 'abc_0' in s])

In [ ]:
import tensorflow as tf
import csv
A = {'learning_rate':0.00009123,'keep_prob':0.25, 'activation':tf.nn.relu, 'channels':[16,16,32]}
# A = {'a':'b','c':'d'}

with open("output"+".csv", "w") as csvfile:
    writer = csv.writer(csvfile)
    for key, val in A.items():
        writer.writerow([key, val])


In [ ]:
from sklearn.datasets import fetch_mldata
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.model_selection import train_test_split
import pandas as pd

In [ ]:
# You can add the parameter data_home to wherever to where you want to download your data
import tensorflow as tf
import numpy as np
((train_data, train_labels),(eval_data, eval_labels)) = tf.keras.datasets.mnist.load_data()
train_data = train_data / np.float32(255)
train_data = train_data.reshape(train_data.shape[0], -1)
print(train_data.shape)
train_labels = train_labels.astype(np.int32)  # not required
print(train_labels.shape)
# test_size: what proportion of original data is used for test set
train_img, test_img, train_lbl, test_lbl = train_test_split(
    train_data, train_labels, test_size=1/7.0, random_state=0)
print(train_img.shape)

In [ ]:
# Standardizing the Data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

# Fit on training set only.
scaler.fit(train_img)

# Apply transform to both the training set and the test set.
train_img = scaler.transform(train_img)
test_img = scaler.transform(test_img)

In [ ]:
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
pca = PCA(.95)
pca.fit(train_img)
pca.n_components_
train_img = pca.transform(train_img)
test_img = pca.transform(test_img)

# all parameters not specified are set to their defaults
# default solver is incredibly slow thats why we change it
# solver = 'lbfgs'
logisticRegr = LogisticRegression(solver = 'lbfgs')

logisticRegr.fit(train_img, train_lbl)

# Returns a NumPy Array
# Predict for One Observation (image)
logisticRegr.predict(test_img[0].reshape(1,-1))

# Predict for Multiple Observations (images) at Once
logisticRegr.predict(test_img[0:10])

In [ ]:
score = logisticRegr.score(test_img, test_lbl)
print(score)

In [ ]:
data_path = './data/preparation_181_data_Taper_Sum_median_eval.tfrecords'
arrays = get_data_from_path(data_path)
# example, label = get_data_from_path(data_path)
# print(example[0])
print((arrays[1]))

In [ ]:
ex, label = get_data_from_path(data_path)
print(label)

In [ ]:
ex, label = get_data_from_path(data_path)
print(label)

In [ ]:
import csv

csvData = [['Person', 'Age'], ['Peter', '22'], ['Jasmine', '21'], ['Sam', '24']]
print(type(csvData))
print((csvData))
print(type(csvData[0]))
print((csvData[0]))
'''
with open('person.csv', 'w') as csvFile:
    writer = csv.writer(csvFile)
    writer.writerows(csvData)

csvFile.close()'''

In [ ]:
csvData = [(-0.53103447, [1e-05, 0.0, '1', 1]), (-0.53103447, [4.315202424267864e-05, 0.0, '1', 1]), (-0.53103447, [6.388961292348475e-05, 0.04720173848853115, '1', 4]), (-0.53103447, [0.0001540877645803547, 0.35419707255064065, '1', 1])]
newData = []
for i in csvData:
    data = {'value': i[0],
           'learning_rate': i[1][0],
           'dropout_rate': i[1][1],
           'activation': i[1][1],
           'channels': i[1][1],}
    newData.append(data)
print(type(csvData))
print((csvData))
#print(type(csvData[0]))
# print((csvData[0]))


with open('person.csv', 'w') as csvFile:
    writer = csv.DictWriter(csvFile, fieldnames = ['value','learning_rate','dropout_rate', 'activation', 'channels'])
    writer.writeheader()
    writer.writerows(newData)



In [ ]:
print("Func vals: %s \n" % search_result.func_vals)
print("Hyper para: %s \n" % search_result.x_iters)
print("Combined: %s" % list(zip(search_result.func_vals, search_result.x_iters)))

In [ ]:
searched_parameter_2 = (list(zip(search_result.func_vals, search_result.x_iters)))
i = searched_parameter_2[0]
data_2 = {field_name[0]: i[0] * -1,
                field_name[1]: i[1][0],
                field_name[2]: i[1][1],
                field_name[3]: i[1][1],
                field_name[4]: i[1][1]}
print(data_2)

In [ ]:
stl_points = points
new_stl_points = []
for stl_point_sample in stl_points:  # stl_point_sample is one example of stl_points
    new_points_sample = []
    for stl_point_image in stl_point_sample:  # stl_point_image is one degree of cross-section
        difference = stl_point_image[1:, :] - stl_point_image[0:-1, :]  # Find difference between each position
        new_points_sample.append(difference)
    new_stl_points.append(new_points_sample)

In [ ]:
print(type(new_stl_points))
print(len(new_stl_points))
print(type(new_stl_points[0]))
print(len(new_stl_points[0]))
print(type(new_stl_points[0][0]))
print(np.shape(new_stl_points[0][0]))

In [ ]:
print(lbl_name)
print(err_name)

In [ ]:
import os
# File name will be [tfrecord_name]_train_Taper_sum_median
tfrecord_file_name = "preparation_181_data"
# tfrecord_name = "original_preparation_data"
# csv_name = "../global_data/Ground Truth Score_50.csv"
# Directory of image
dataset_folder_dir = "./data/cross_section"
 
file_name = './data/preparation_181_data_train_address.txt'
name = []
if os.path.isfile(file_name):  # Check if file exist
    with open(file_name) as f:
        name = f.read().splitlines()
    '''
    with open(file_name, 'r') as filehandle:
        for line in filehandle:
            name.append(line)
            
            # remove linebreak which is the last character of the string
            # current_name = line[:-1]
            
            # check if it exist in grouped exist, if found, put in train_address
            for i, name in enumerate(example_grouped_address):
                if current_name in name:
                    train_address.append(grouped_address[i])
                    grouped_address.remove(grouped_address[i])
                    example_grouped_address.remove(example_grouped_address[i])
                    break
            '''
    # print("Use %s examples from previous tfrecords as training" % len(train_address))
else:
    print("Not found")

In [ ]:
print(type(name))
for n in name[1:]:
    print(n)

In [ ]:
print (n[1] == '/home/pasin/Documents/Google_Drive/Aa_TIT_LAB_Comp/Library/Tooth/Tooth/Model/my2DCNN/data/cross_section/PreparationScan_1_85148')

In [ ]:
import numpy as np
import tensorflow as tf
A = np.array([[1,2],[3,4],[5,6]])
B = tf.cast(A,tf.float32)
print(B)
sess = tf.Session()
print(sess.run(B))

In [ ]:
def readjust_median_label(label, avg_data):
    possible_value = [1,3,5]
    for i, label_value in enumerate(label):
        if not (label_value in possible_value):
            # Check if value is over/under boundary, if so, choose the min/max value
            if label_value < possible_value[0]:
                label[i] = possible_value[0]
            elif label_value > possible_value[-1]:
                label[i] = possible_value[-1]
            else:
                if label_value > avg_data[i]:  # If median is more than average, round up
                    label[i] = min(filter(lambda x: x > label_value, possible_value))
                else:  # If median is less or equal to average, around down
                    label[i] = max(filter(lambda x: x < label_value, possible_value))
    return label

A = [1,3,5,2,2,4,4, 10, -2, 2]
B = [1,4,2,3,1,4,7, 15]

C = readjust_median_label(A,B)
print(C)

In [ ]:
import numpy as np
data = np.load('/home/pasin/Documents/Google_Drive/Aa_TIT_LAB_Comp/Library/Tooth/Tooth/Model/my2DCNN/data/coordinates/PreparationScan_1_305139_0.npy')
print(type(data))
print(data)

In [ ]:
import numpy as np
A = {'A':11,'B':21}
print(len(A))
C = [3,4]
for i,(key,value) in enumerate(A.items()):
    print(i)
    print(type(key))
    print(type(value))

In [ ]:
def testFunction(**args):
    B = {}
    for key,value in args.items():
        print(key)
        print(type(key))
        print(value)
        print(type(value))
        B[key] = value
    return B
B = testFunction(c=1,b=2,a =3)
print(B)

In [ ]:
def check_exist(dictionary, **kwargs):
    output_dict = dictionary
    for key, value in kwargs.items():
        try:
            output_dict[key] = dictionary[key]
            # output = params[dict_name]
        except (KeyError, TypeError) as error:
            if value is None:
                raise Exception("Parameter '%s' not defined" % key)
            else:
                output_dict[key] = value
                print("Parameters: %s not found, use default value = %s" % (key, value))
    return output_dict

X = {'A':11,'B':21}
Y = check_exist(X,A=15,B=None,C=50)
print(Y)

In [ ]:
A = [1,2,3]
B = [4,5]
print(A.expand(B))

In [ ]:
import glob

file_name = []
for file in glob.glob('/home/pasin/Documents/Pasin/model/tooth_max_pooling_regression_50/hyperparameters_result_*'):
    file_name.append(file)
file_name.sort()
print(file_name)

In [ ]:
import csv
def read_file(csv_dir, header = False):
    header_name = []
    data = []
    with open(csv_dir) as csvfile:
        readCSV = csv.reader(csvfile, delimiter=',')
        for row in readCSV:
            print(row)
            if header:
                header_name.append(row)
                header = False
            else:
                data.append(row)
    if not header_name:
        return data
    else:
        return data, header_name

In [ ]:
data, header = read_file(file_name[-2], True)
print(header)
print("\n")
print(data[-2])
l_data = data[-2][1:]
print([float(l_data[0]), float(l_data[1]),l_data[2],int(l_data[3]),int(l_data[3])])

In [ ]:
import tensorflow as tf
import numpy as np

labels = tf.constant([0,1,2,0])
labels_one_hot = tf.one_hot(labels,3)

weight = tf.constant([[1,2,3]], dtype=tf.float32)
new_w = tf.matmul(labels_one_hot,weight,transpose_b=True)

logits = tf.constant([[0,1,0],[0,1,0],[0,1,0],[1,0,0]], dtype  =tf.float32)

loss = tf.losses.sparse_softmax_cross_entropy(labels, logits, weights = new_w)

sess = tf.Session()

print(sess.run(loss))

In [ ]:
import numpy as np
import tensorflow as tf
def read_npy_file(item):
    print("Item: %s" % item)
    label = item[1]
    image = []
    for i in range(4):
        img = np.load(item[0][i])
        image.append(img.astype(np.float32))
    return (image, label)

A = "/home/pasin/Documents/Link to Tooth/Tooth/Model/my2DCNN/data/coordinate_300_point/"
B = tf.py_function(read_npy_file, [A], [tf.float32, ])

sess = tf.Session()
print(sess.run(B))

In [ ]:
import glob
previous_record_files = []
for file in glob.glob('/home/pasin/Documents/Pasin/model/BL_classification' + '/' + "hyperparameters_result_" + '*'):
    previous_record_files.append(file)
print(previous_record_files)

In [ ]:
from open_save_file import save_file

field_name = ['accuracy', 'learning_rate', 'dropout_rate', 'activation', 'cnn_channels', 'fc_channels']
save_file('/home/pasin/Documents/Pasin/model/BL_classification/hyperparameters_result_20190424_000.csv', [], field_name = field_name)

In [ ]:
from skopt.space import Real, Categorical, Integer
dim_learning_rate = Real(low=1e-5, high=1e-2, prior='log-uniform', name='learning_rate')
dim_dropout_rate = Real(low=0, high=0.875, name='dropout_rate')
dim_activation = Categorical(categories=['0', '1'],
                             name='activation')
dim_channel = Integer(low=1, high=4, name='channels')
dim_channel_fc = Integer(low=1, high=2, name='fully_connect_channels')  # Fully conencted
dim_loss_weight = Real(low=0.8, high=2, name='loss_weight')
dimensions = [dim_learning_rate,
              dim_dropout_rate,
              dim_activation,
              dim_channel,
              dim_channel_fc,
              dim_loss_weight]

dim_name = [i.name for i in dimensions]
print(dim_name)

In [ ]:
import tensorflow as tf

numdegree = 4

def create_file_reader_ops(filename_queue):
    reader = tf.TextLineReader(skip_header_lines=1)
    _, csv_row = reader.read(filename_queue)
    record_defaults = [[0], [0]]
    x,y = tf.decode_csv(csv_row, record_defaults=record_defaults)
    features = tf.pack([x,y])
    return features


# Import tfrecord to dataset
def deserialize(example):
    feature = {'label': tf.FixedLenFeature([], tf.int64)}
    for i in range(numdegree):
        feature['img' + str(i)] = tf.FixedLenFeature([], tf.string)
    return tf.parse_single_example(example, feature)


def decode(data_dict):
    if numdegree != 4:
        raise Exception('Number of degree specified is not compatible, edit code')
    # Create initial image, then stacking it
    image_decoded = []

    # Stacking the rest
    for i in range(0, numdegree):
        img = data_dict['img' + str(i)]
        file_name = tf.read_file(img)
        filename_queue = tf.train.string_input_producer([file_name])
        example = create_file_reader_ops(filename_queue)
        image_decoded.append(example)
    
    # image_stacked = tf.stack([image_decoded[0], image_decoded[1], image_decoded[2], image_decoded[3]], axis=2)
    # image_stacked = tf.cast(image_stacked, tf.float32)
    label = tf.cast(data_dict['label'], tf.float32)
    # output = (image_stacked, label)
    # return {'images': image_stacked, 'label': label}  # Output is [Channel, Height, Width]
    return image_decoded[0], label# image_stacked, label

data_path = "/home/pasin/Documents/Link to Tooth/Tooth/Model/my2DCNN/data/tfrecord/preparation_coor_debug_BL_median_coor_train.tfrecords"
dataset = tf.data.TFRecordDataset(data_path)
dataset = dataset.map(deserialize, num_parallel_calls=7)
dataset = dataset.map(decode, num_parallel_calls=7)

iterator = dataset.make_one_shot_iterator()
next_image_data = iterator.get_next()
images = []
label = []
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    try:
        # Keep extracting data till TFRecord is exhausted
        while True:
            data = sess.run(next_image_data)
            images.append(data[0])
            label.append(data[1])
    except tf.errors.OutOfRangeError:
        pass    

In [ ]:
def create_file_reader_ops(filename_queue):
    reader = tf.TextLineReader(skip_header_lines=1)
    _, csv_row = reader.read(filename_queue)
    record_defaults = [[0], [0]]
    x,y = tf.decode_csv(csv_row, record_defaults=record_defaults)
    features = tf.pack([x,y])
    return features
filename_queue = tf.train.string_input_producer(["/home/pasin/Documents/Link to Tooth/Tooth/Model/my2DCNN/data/tfrecord/preparation_coor_debug_BL_median_coor_train.tfrecords"], num_epochs=1, shuffle=False)
example = create_file_reader_ops(filename_queue)
print(example)

In [ ]:
A = [1,2,3]
print(dict(x=5,y=0))

In [ ]:
import numpy as np
A = np.array([[1,2],[3,4],[5,6],[7,8],[9,10]])
B = tf.sparse_tensor_to_dense(A,default_value=0)
C= tf.reshape(B,[10])

sess = tf.Session()

print(sess.run(B))
print(sess.run(C))

In [ ]:
import tensorflow as tf
# Read TFRecord file
reader = tf.TFRecordReader()
filename_queue = tf.train.string_input_producer(['/home/pasin/Documents/Link to Tooth/Tooth/Model/my2DCNN/data/tfrecord/preparation_coor_smallraw_Taper_Sum_median_coor_train.tfrecords'])

_, serialized_example = reader.read(filename_queue)
numdegree = 4
# Define features
def deserialize(example):
    feature = {'label': tf.FixedLenFeature([], tf.int64)}
    for i in range(numdegree):
        for j in range(2):
            feature['img' + str(i) + str(j)] = tf.FixedLenFeature([], tf.float32)
    return tf.parse_single_example(example, feature)

# Extract features from serialized data
read_data = deserialize(filename_queue)

# Many tf.train functions use tf.train.QueueRunner,
# so we need to start it before we read
tf.train.start_queue_runners(sess)

# Print features
with tf.Session() as sess:
    for name, tensor in sess.run(read_data).items():
        print('{}: {}'.format(name, tensor))

In [ ]:
from get_data import get_data_from_path



p = "/home/pasin/Documents/Link to Tooth/Tooth/Model/my2DCNN/data/tfrecord/preparation_361_MD_median_train.tfrecords"
img = get_data_from_path(p)

In [3]:
from open_save_file import get_label
import collections
label, label_name = get_label("BL","median",double_data=True,one_hotted=False,normalized=False)

counter=collections.Counter(label)
for k,v in counter.items():
    print(k)
    print(v)
print(counter)

get_label: Import from ../global_data/Ground Truth Score_new.csv
get_label: Upload non one-hotted label completed (as a list): 730 examples
3
450
5
264
1
16
Counter({3: 450, 5: 264, 1: 16})


In [6]:
import tensorflow as tf

weight = tf.constant([[1,2, 1]],dtype=tf.float32)
loss_weight = tf.matmul(one_hot_label,weight,transpose_b=True,a_is_sparse=True)

3
150
